In [1]:
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import mean_squared_error, r2_score

from skorch import NeuralNetRegressor

In [2]:
random_state = 177013
torch.manual_seed (random_state)
batch_size = 1000
num_workers = 4

In [3]:
df = pd.read_csv('auto_filtered.csv', index_col=0)
df.head()

,Price,VehicleType,RegistrationYear,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,PostalCode,Gearbox_manual,NotRepaired_yes
3,1500,small,2001,75,golf,150000,6,petrol,volkswagen,91074,1,0
4,3600,small,2008,69,fabia,90000,7,petrol,skoda,60437,1,0
5,650,sedan,1995,102,3er,150000,10,petrol,bmw,33775,1,1
6,2200,convertible,2004,109,2_reihe,150000,8,petrol,peugeot,67112,1,0
10,2000,sedan,2004,105,3_reihe,150000,12,petrol,mazda,96224,1,0


In [4]:
df = pd.get_dummies(df, columns = ['VehicleType', 'Model', 'FuelType', 'Brand'], drop_first=True)

In [5]:
df_features = df.drop(['Price','Gearbox_manual','RegistrationMonth'], axis=1)
df_target = df['Price']

In [6]:
features_train, features_test, target_train, target_test = train_test_split(df_features, df_target,
                                                                            test_size=0.2, random_state=random_state)

In [7]:
quantile_transformer = QuantileTransformer(random_state=random_state)
quantile_transformer.fit(features_train)
features_train = quantile_transformer.transform(features_train)
features_test = quantile_transformer.transform(features_test)

In [8]:
class Linear(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.fc1 = torch.nn.Linear(n_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu1 = nn.LeakyReLU()
        self.relu2 = nn.LeakyReLU()
        self.norm1 = nn.BatchNorm1d(64)
        self.norm2 = nn.BatchNorm1d(32)

    def forward(self, batch):
        batch = self.fc1(batch)
        batch = self.relu1(batch)
        batch = self.norm1(batch)
        batch = self.fc2(batch)
        batch = self.relu2(batch)
        batch = self.norm2(batch)
        return self.fc3(batch)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

The model will be running on cpu device


In [10]:
model = Linear(df_features.shape[1])

In [11]:
net = NeuralNetRegressor(
    model,
    max_epochs=25,
    batch_size=batch_size,
    lr=0.005,
    criterion = nn.MSELoss,
    optimizer = Adam,
    device = device,
)

In [12]:
net.fit(features_train.astype('float32'), target_train.values.astype('float32').reshape(-1, 1))

  epoch     train_loss     valid_loss     dur
-------  -------------  -------------  ------
      1  17351227.4510  17189124.7717  1.4562
      2  16451081.7891  15871585.9428  1.3413
      3  14755546.5602  13862332.4786  1.3430
      4  12509712.0399  11362660.1687  1.3157
      5  10038213.4093  8779011.4692  1.3073
      6  7656945.2794  6532941.8336  1.3716
      7  5608395.3989  4608926.0222  1.4055
      8  4020800.7274  3259061.3750  1.3737
      9  2897382.7267  2360714.6933  1.3610
     10  2158731.5422  1858243.2549  1.3605
     11  1693438.9412  1558393.7344  1.3513
     12  1400690.8189  1337812.7334  1.3283
     13  1227819.0453  1222948.4728  1.6220
     14  1137469.0805  1166555.4785  1.6119
     15  1093434.7716  1140800.5412  1.6147
     16  1072815.1502  1131025.2237  1.6465
     17  1063582.7557  1128272.8487  1.6166
     18  1057614.6655  1126299.3918  1.6139
     19  1053580.4701  1124610.2206  1.4415
     20  1050718.1290  1124579.7891  1.3490
     21  1047684.29

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Linear(
    (fc1): Linear(in_features=301, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=32, bias=True)
    (fc3): Linear(in_features=32, out_features=1, bias=True)
    (relu1): LeakyReLU(negative_slope=0.01)
    (relu2): LeakyReLU(negative_slope=0.01)
    (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (norm2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  ),
)

In [13]:
predictions = net.predict(features_test.astype('float32'))
mse = mean_squared_error(target_test, predictions)
r2 = r2_score(target_test, predictions)
print(f'Test RMSE: {mse ** 0.5:.2f}, R2: {r2:.2f}')

Test RMSE: 1058.90, R2: 0.82
